# Trained and test set predictions for mt5 small embeddings for chichewa and english

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project/

/content/gdrive/MyDrive/Masters/Year_1/Semester_2/CS_505/Project


In [ ]:
chichewa_embd = np.load( 'Data/' + 'embeddings_chichewa_512_final.npy')

In [ ]:
chichewa_embd[0][:10]

array([ 0.01402206, -0.09882534,  0.01521077, -0.14700669, -0.04015165,
        0.05069736,  0.24382819,  0.05384792,  0.07545777,  0.0441238 ],
      dtype=float32)

In [ ]:
english_embd = np.load('Data/' + 'embeddings_english_512_final.npy')

In [ ]:
english_embd[0][:10]

array([ 0.10436802,  0.00451875,  0.00795077, -0.03812902,  0.0005198 ,
        0.08021696,  0.2597164 , -0.04285967,  0.06787069, -0.0873456 ],
      dtype=float32)

In [ ]:
final_comb_arr = np.hstack((chichewa_embd,english_embd))

In [ ]:
train_df = pd.read_csv('Data/Train.csv')

In [ ]:
final_comb_arr.shape

(1436, 1024)

In [ ]:
le = LabelEncoder()
train_df.Label = le.fit_transform(train_df.Label)

In [ ]:
le.classes_

array(['ARTS AND CRAFTS', 'CULTURE', 'ECONOMY', 'EDUCATION', 'FARMING',
       'FLOODING', 'HEALTH', 'LAW/ORDER', 'LOCALCHIEFS', 'MUSIC',
       'OPINION/ESSAY', 'POLITICS', 'RELATIONSHIPS', 'RELIGION', 'SOCIAL',
       'SOCIAL ISSUES', 'SPORTS', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT',
       'WITCHCRAFT'], dtype=object)

In [ ]:
Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [ ]:
def get_model(size):
  print('Building model...')
  model = Sequential()
  model.add(Dense(2048, input_shape=(size,)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(20))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
  #make one, takes in, double that amount, english and chichewa

In [ ]:
print(Y_train.shape)
print(train_df.shape)

(1436, 20)
(1436, 3)


In [ ]:
test_df = pd.read_csv('Data/Test.csv')

In [ ]:
test_df.head()

,ID,Text
0,ID_ADHEtjTi,Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1,ID_AHfJktdQ,Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2,ID_AUJIHpZr,Anatcheleza: Akundiopseza a gogo wanga Akundi...
3,ID_AUKYBbIM,Ulova wafika posauzana Adatenga digiri ya uph...
4,ID_AZnsVPEi,"Dzombe kukoma, koma Kuyambira makedzana, pant..."


In [ ]:
#Y_train = np_utils.to_categorical(train_df['Label'], np.unique(train_df['Label']).shape[0])

In [ ]:
#print(train_df.shape)
#print(Y_train.shape)

In [ ]:
#tfidf = TfidfVectorizer()
#tfidf_train = tfidf.fit_transform(train_df.Text).toarray()
#tfidf_test = tfidf.transform(X_test).toarray()
train_dataset = tf.data.Dataset.from_tensor_slices((final_comb_arr, Y_train))

#test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test, y_test))

BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#test_dataset = test_dataset.batch(BATCH_SIZE)
model_2 = get_model(final_comb_arr.shape[1])

model_2.fit(train_dataset,epochs=100, batch_size=32,verbose=1)

Building model...
Epoch 1/100
45/45 [==============================] - 2s 5ms/step - loss: 2.3903 - accuracy: 0.2730
Epoch 2/100
45/45 [==============================] - 0s 5ms/step - loss: 1.9226 - accuracy: 0.4318
Epoch 3/100
45/45 [==============================] - 0s 4ms/step - loss: 1.6915 - accuracy: 0.4847
Epoch 4/100
45/45 [==============================] - 0s 5ms/step - loss: 1.5718 - accuracy: 0.5139
Epoch 5/100
45/45 [==============================] - 0s 5ms/step - loss: 1.4793 - accuracy: 0.5446
Epoch 6/100
45/45 [==============================] - 0s 5ms/step - loss: 1.4072 - accuracy: 0.5599
Epoch 7/100
45/45 [==============================] - 0s 4ms/step - loss: 1.3297 - accuracy: 0.5787
Epoch 8/100
45/45 [==============================] - 0s 4ms/step - loss: 1.2721 - accuracy: 0.5940
Epoch 9/100
45/45 [==============================] - 0s 6ms/step - loss: 1.2198 - accuracy: 0.6065
Epoch 10/100
45/45 [==============================] - 0s 4ms/step - loss: 1.1646 - accuracy

In [ ]:
chichewa_embd_test = np.load('Data/' + 'embeddings_chichewa_512_final_test.npy')

In [ ]:
chichewa_embd_test[0][:10]

array([ 0.02143669, -0.11324774,  0.03792359, -0.18134049, -0.05576902,
        0.04920467,  0.2680038 ,  0.05318359,  0.05776734, -0.07490677],
      dtype=float32)

In [ ]:
english_embd_test = np.load('Data/' + 'embeddings_english_512_final_test.npy')

In [ ]:
english_embd_test[0][:10]

array([ 0.12655264, -0.04361302, -0.00195987, -0.03778377, -0.05715036,
        0.09073132,  0.2840523 ,  0.00882943, -0.02536617, -0.17974791],
      dtype=float32)

In [ ]:
final_comb_arr_test = np.hstack((chichewa_embd_test,english_embd_test))

In [ ]:
final_comb_arr_test.shape

(620, 1024)

In [ ]:
#mt5_test = tfidf.transform(test_df.Text).toarray()
#test_dataset = tf.data.Dataset.from_tensor_slices((tfidf_test))

In [ ]:
print(test_df.shape)

(620, 2)


In [ ]:
ans = model_2.predict(final_comb_arr_test)

In [ ]:
print(ans.shape)
ans_arr = np.argmax(ans,axis=1)

(620, 20)


In [ ]:
ans_arr[:2]

array([15, 13])

In [ ]:
le.classes_

array(['ARTS AND CRAFTS', 'CULTURE', 'ECONOMY', 'EDUCATION', 'FARMING',
       'FLOODING', 'HEALTH', 'LAW/ORDER', 'LOCALCHIEFS', 'MUSIC',
       'OPINION/ESSAY', 'POLITICS', 'RELATIONSHIPS', 'RELIGION', 'SOCIAL',
       'SOCIAL ISSUES', 'SPORTS', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT',
       'WITCHCRAFT'], dtype=object)

In [ ]:
arr_class = list(le.inverse_transform(ans_arr))

In [ ]:
arr_class[:5]

['SOCIAL ISSUES', 'RELIGION', 'SOCIAL ISSUES', 'POLITICS', 'SOCIAL ISSUES']

In [ ]:
test_df_copy = test_df.copy()

In [ ]:
print(test_df_copy)

              ID                                               Text
0    ID_ADHEtjTi   Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1    ID_AHfJktdQ  Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2    ID_AUJIHpZr   Anatcheleza: Akundiopseza a gogo wanga Akundi...
3    ID_AUKYBbIM   Ulova wafika posauzana Adatenga digiri ya uph...
4    ID_AZnsVPEi   Dzombe kukoma, koma Kuyambira makedzana, pant...
..           ...                                                ...
615  ID_zdpOUWyJ  Kanyongolo Wapempha Oyimira Milandu Kuti Atsat...
616  ID_zhnOomuu  Amandimenya\nZikomo gogo,\nNdine mtsikana wa z...
617  ID_zmWHvBJb   Apolisi athotha gulu la MYP Asilikali 56 a gu...
618  ID_zphjdFIb   Mwambo wa ukwati wa Chitonga Mtundu wina uliw...
619  ID_ztdtrNxt   Mwapasa autsa mapiri Pamene pali kusamvana pa...

[620 rows x 2 columns]


In [ ]:
test_df_copy["Labels"] = arr_class

In [ ]:
print(test_df_copy)

              ID                                               Text  \
0    ID_ADHEtjTi   Abambo odzikhweza akuchuluka Kafukufuku wa ap...   
1    ID_AHfJktdQ  Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...   
2    ID_AUJIHpZr   Anatcheleza: Akundiopseza a gogo wanga Akundi...   
3    ID_AUKYBbIM   Ulova wafika posauzana Adatenga digiri ya uph...   
4    ID_AZnsVPEi   Dzombe kukoma, koma Kuyambira makedzana, pant...   
..           ...                                                ...   
615  ID_zdpOUWyJ  Kanyongolo Wapempha Oyimira Milandu Kuti Atsat...   
616  ID_zhnOomuu  Amandimenya\nZikomo gogo,\nNdine mtsikana wa z...   
617  ID_zmWHvBJb   Apolisi athotha gulu la MYP Asilikali 56 a gu...   
618  ID_zphjdFIb   Mwambo wa ukwati wa Chitonga Mtundu wina uliw...   
619  ID_ztdtrNxt   Mwapasa autsa mapiri Pamene pali kusamvana pa...   

            Labels  
0    SOCIAL ISSUES  
1         RELIGION  
2    SOCIAL ISSUES  
3         POLITICS  
4    SOCIAL ISSUES  
..             ...  


In [ ]:
subm = pd.read_csv('Data/SampleSubmission.csv')

In [ ]:
subm.head()

,ID,Label
0,ID_sQaPRMWO,0
1,ID_TanclvfR,0
2,ID_CNbveyvk,0
3,ID_MclKMhyP,0
4,ID_rNrmXOGD,0


In [ ]:
subm.drop(columns=['Label'],inplace=True)

In [ ]:
n_df = pd.merge(subm,test_df_copy, left_on='ID', right_on='ID')

In [ ]:
n_df.rename(columns={"Labels": "Label"},inplace=True)

In [ ]:
n_df.head()

,ID,Text,Label
0,ID_sQaPRMWO,Two Sentenced to 5 Years for Armed Robbery Man...,LAW/ORDER
1,ID_TanclvfR,Papa Wakhazikitsa Bungwe Lolimbikitsa Umodzi W...,RELIGION
2,ID_CNbveyvk,Amalawi a nsomba ali kakasi ku Botswana Ali m...,SOCIAL ISSUES
3,ID_MclKMhyP,Mtumiki wa Mulungu adalosera Onse okhulupirir...,RELIGION
4,ID_rNrmXOGD,Kubwera kwa 1 000 Kwacha kukusonyeza sizikuye...,POLITICS


In [ ]:
n_df.to_csv('Data/MLP_MT5_100_Submission.csv',columns=['ID','Label'],index=False)